In [1]:
from finn.util.visualization import showInNetron
from finn.core.modelwrapper import ModelWrapper

build_dir = "/workspace/finn/notebooks/dance-dance/onnx"

In [2]:
showInNetron(f"{build_dir}/dance-dance.onnx")

Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance.onnx' at http://0.0.0.0:8081


### Tidy-up transformations

In [3]:
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveUniqueNodeNames, GiveReadableTensorNames, RemoveStaticGraphInputs
from finn.transformation.infer_datatypes import InferDataTypes

model = ModelWrapper(f"{build_dir}/dance-dance.onnx")

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(f"{build_dir}/dance-dance_tidy.onnx")
showInNetron(f"{build_dir}/dance-dance_tidy.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance_tidy.onnx' at http://0.0.0.0:8081


### Streamlining

In [4]:
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
from finn.transformation.streamline import Streamline

model = ModelWrapper(f"{build_dir}/dance-dance_tidy.onnx")

model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())

model.save(f"{build_dir}/dance-dance_streamlined.onnx")
showInNetron(f"{build_dir}/dance-dance_streamlined.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance_streamlined.onnx' at http://0.0.0.0:8081


### Conversion to HLS layers

In [5]:
from finn.transformation.fpgadataflow.convert_to_hls_layers import InferQuantizedStreamingFCLayer

model = ModelWrapper(f"{build_dir}/dance-dance_streamlined.onnx")

model = model.transform(InferQuantizedStreamingFCLayer("decoupled"))

model.save(f"{build_dir}/dance-dance_hls.onnx")
showInNetron(f"{build_dir}/dance-dance_hls.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance_hls.onnx' at http://0.0.0.0:8081


### Create dataflow partition

In [6]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper(f"{build_dir}/dance-dance_hls.onnx")

model = model.transform(CreateDataflowPartition())

model.save(f"{build_dir}/dance-dance_dataflow_parent.onnx")
showInNetron(f"{build_dir}/dance-dance_dataflow_parent.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance_dataflow_parent.onnx' at http://0.0.0.0:8081


### Folding

In [7]:
from finn.custom_op.registry import getCustomOp

parent_model = ModelWrapper(f"{build_dir}/dance-dance_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
model = ModelWrapper(dataflow_model_filename)

fc_layers = model.get_nodes_by_op_type("StreamingFCLayer_Batch")

# (PE, SIMD, in_fifo_depth, out_fifo_depth, ramstyle) for each layer
# 1: 81, 240
# 2: 9, 81
config = [
    (1, 1, 1, 1, "block"),
    (1, 1, 1, 1, "block"),
]

for fcl, (pe, simd, ififo, ofifo, ramstyle) in zip(fc_layers, config):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepth", ififo)
    fcl_inst.set_nodeattr("outFIFODepth", ofifo)
    fcl_inst.set_nodeattr("ram_style", ramstyle)

model.save(f"{build_dir}/dance-dance_fold.onnx")
showInNetron(f"{build_dir}/dance-dance_fold.onnx")


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/notebooks/dance-dance/onnx/dance-dance_fold.onnx' at http://0.0.0.0:8081


### Build hardware

In [8]:
from finn.util.basic import pynq_part_map
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

pynq_board = "Ultra96"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

model = ModelWrapper(f"{build_dir}/dance-dance_fold.onnx")
model = model.transform(ZynqBuild(platform=pynq_board, period_ns=target_clk_ns))

model.save(f"{build_dir}/dance-dance_synthesis.onnx")

/workspace/finn/src/finn/transformation/infer_data_layouts.py:118: UserWarning: Assuming 2D input is NC
  warnings.warn("Assuming 2D input is NC")


### View resources utilisation

In [9]:
from finn.transformation.fpgadataflow.annotate_resources import AnnotateResources

model = ModelWrapper(f"{build_dir}/dance-dance_synthesis.onnx")

model = model.transform(AnnotateResources(mode="synth"))
model.get_metadata_prop("res_total_top_synth")

"{'LUT': 8841, 'SRL': 242, 'FF': 13123, 'BRAM_36K': 5, 'BRAM_18K': 3, 'DSP48': 0}"

### PYNQ deployment

In [10]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()

SUNFIRE_USER = getenv("SUNFIRE_USER")
SUNFIRE_PW = getenv("SUNFIRE_PW")
ULTRA96_IP = getenv("ULTRA96_IP")
ULTRA96_PORT = getenv("ULTRA96_PORT")
ULTRA96_USER = getenv("ULTRA96_USER")
ULTRA96_PW = getenv("ULTRA96_PW")

In [15]:
!sshpass -p {SUNFIRE_PW} ssh -NfL {ULTRA96_PORT}:{ULTRA96_IP}:22 {SUNFIRE_USER}@sunfire.comp.nus.edu.sg

In [16]:
from finn.transformation.fpgadataflow.make_deployment import DeployToPYNQ

ip = "localhost"
port = ULTRA96_PORT
username = ULTRA96_USER
password = ULTRA96_PW
target_dir = "/home/xilinx/dance-dance/ann"

model = ModelWrapper(f"{build_dir}/dance-dance_synthesis.onnx")

model = model.transform(DeployToPYNQ(ip, port, username, password, target_dir))

model.save(f"{build_dir}/dance-dance_deploy.onnx")

In [17]:
model.get_metadata_prop("pynq_deployment_dir")

'/tmp/finn_dev_vernon/pynq_deployment_r20yawgs'

### Throughput test

In [18]:
from finn.core.throughput_test import throughput_test_remote

model = ModelWrapper(f"{build_dir}/dance-dance_deploy.onnx")

res = throughput_test_remote(model, 1)
print("Network metrics:")
for key in res:
    print(f"{key}: {res[key]}")

Network metrics:
runtime[ms]: 0.39958953857421875
throughput[images/s]: 2502.568019093079
DRAM_in_bandwidth[Mb/s]: 0.1501540811455847
DRAM_out_bandwidth[Mb/s]: 0.04504622434367542
fclk[mhz]: 99.999
N: 1
